In [26]:
class CNF:
    def __init__(self, n_var, n_clauses, clauses):
        self.var_map = {}
        self.n_var = n_var
        self.n_clauses = n_clauses
        self.clauses = clauses
        self.satisfiable = 0 # unknown -> 0, sat -> 1, unsat -> 2

In [27]:
def make_formula(filename):
    f = open(filename)
    lines = f.readlines()
    flag = 0
    clauses = []
    n_var = 0
    n_clauses = 0
    for line in lines:
        if (flag == 0):
            if (line[0] == 'p'):
                flag = 1
                info = line.split()
                n_var = info[2]
                n_clauses = info[3]
        else:
            clause = list(map(int, line.split()[:-1]))
            clauses.append(clause)
    return CNF(n_var, n_clauses, clauses)


In [28]:
cnf = make_formula('test1.cnf')
print(cnf.clauses)

[[1, -5, 4], [-1, 5, 3, 4], [-4]]


In [29]:
def unit_var(cnf):
    clauses = cnf.clauses
    new_clauses = []
    unit_var = 0
    unit_index = []
    for i, clause in enumerate(clauses):
        if (len(clause) == 1):
            unit_var = clause[0]
            if (unit_var < 0):
                if not (check_var_map(cnf.var_map, -unit_var, 0)):
                    cnf.satisfiable = 2
                cnf.var_map[-unit_var] = 0
            else:
                if not (check_var_map(cnf.var_map, unit_var, 1)):
                    cnf.satisfiable = 2
                cnf.var_map[unit_var] = 1
                unit_index.append(i)
    for i in range(len(clauses)):
        if not (i in unit_index):
            new_clauses.append(clauses[i])
    cnf.clauses = new_clauses
    cnf.n_clauses = len(new_clauses)
    if (cnf.n_clauses == 0):
        cnf.satisfiable = 1
    return cnf

In [30]:
def check_var_map(a, var, val):
    if (var in a):
        if (a[var] != val):
            return False
        else:
            return True
    else:
        return True

In [31]:
f = open('test1.cnf')
lines = f.readlines()
flag = 0
clauses = []
for i, line in enumerate(lines):
    if (flag == 0):
        print(line[0])
        if (line[0] == 'p'):
            flag = 1
            k = line.split()
            print(k)
    else:
        clause = list(map(int, line.split()))
        clause = clause[:-1]
        clauses.append(clause)
print(clauses)
        

c
c
p
['p', 'cnf', '5', '3']
[[1, -5, 4], [-1, 5, 3, 4], [-4]]
